In [11]:
from datetime import datetime

# print date as date accessed
date_accessed = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Date accessed: {date_accessed}")

Date accessed: 2024-10-12 10:12:47


In [2]:
import xarray as xr
import dask
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os, sys, glob, re, time, math, calendar

# import custom functions
sys.path.append('/')
from libraries import *

In [3]:
station_id = 'PROF_OWEG'
ds = xr.open_dataset(f'data/NYSM_standard_and_profiler_combined_wind_speed/{station_id}.nc')
data = ds.wind_speed.sel(range=slice(10,500))
data

<xarray.DataArray 'wind_speed' (range: 18, time: 52560)> Size: 4MB
[946080 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 420kB 2018-01-02 ... 2024-01-01
  * range    (range) int64 144B 10 100 125 150 175 200 ... 400 425 450 475 500
    station  <U9 36B ...

- The idea is, not all instances have wind speed values at all vertical levels. 
- During some instances, wind speed at all the vertical levels might be nans. 
- During some instances, wind speed at some vertical levels might be nans, while the non-missing are insufficient for a complete profile building. 
- During some instances, wind speed at some vertical levels might be nans, while the non-missing are sufficient for a complete profile building. 
- In such scenarios, we need provide the vertical levels having non-missing values in computing the Chebyshev coefficients. 

<xarray.DataArray 'wind_speed' ()> Size: 8B
np.int64(29581)
Coordinates:
    station  <U9 36B ... non-missing instances
Bias: -0.0000
MAE: 0.0855
RMSE: 0.1115
Rmax: 0.2815


In [37]:
# find data only with non-missing instances
station_id = 'PROF_OWEG'
ds = xr.open_dataset(f'data/NYSM_standard_and_profiler_combined_wind_speed/{station_id}.nc')
data = ds.wind_speed.sel(range=slice(10,500))

non_missing_mask = data.notnull().all(dim='range')
non_missing_data = data.where(non_missing_mask, drop=True)
print(non_missing_mask.sum(dim='time'), 'non-missing instances')

# randomly select one profile for visualization
np.random.seed(14000)
time_idx = np.random.choice(non_missing_data.time)
M = non_missing_data.sel(time=time_idx)
Z = M.range.values

ref_H = np.array([10] + list(range(100, 501, 25)))
poly_order = 4
CPtype = 1
Coeff = Chebyshev_Coeff(Z, M.values,poly_order=poly_order, CPtype=CPtype, ref_H=ref_H)
Mp = WindProfile(Z,Coeff, poly_order=poly_order,CPtype=CPtype,ref_H=ref_H)
# Calculate statistics
Bias = np.mean(M - Mp)
MAE = np.mean(np.abs(M - Mp))
RMSE = np.sqrt(np.mean((M - Mp)**2))
Rmax = np.max(np.abs(M - Mp))

# Print the statistics
print(f'Bias: {Bias:.4f}')
print(f'MAE: {MAE:.4f}')
print(f'RMSE: {RMSE:.4f}')
print(f'Rmax: {Rmax:.4f}')

# Create Coeff_tot array
Coeff_tot = np.zeros(10)
Coeff_tot[:len(Coeff)] = Coeff

# Plot and save the figure
fig, ax = plt.subplots()
ax.plot(M, Z, 'or', markerfacecolor='r')
ax.plot(Mp, Z, '-k', linewidth=2)
ax.set(xlabel='Wind Speed (m/s)', ylabel='Height (m)')
ax.set_xlim([0, 30])
ax.set_ylim([0, 500])
ax.tick_params(labelsize=14)
ax.legend(['Measured', 'Modeled'])
plt.text(20, 400, f'C0 = {Coeff_tot[0]:.2f}')
plt.text(20, 350, f'C1 = {Coeff_tot[1]:.2f}')
plt.text(20, 300, f'C2 = {Coeff_tot[2]:.2f}')
plt.text(20, 250, f'C3 = {Coeff_tot[3]:.2f}')
plt.text(20, 200, f'C4 = {Coeff_tot[4]:.2f}')
plt.text(20, 150, fr'mean$_{{measured}}$ = {M.mean():.3f}')
plt.text(20, 100, fr'mean$_{{modelled}}$ = {Mp.mean():.3f}')
plt.savefig(f'data/Figures/chebyshev_fit_with_p{poly_order}_no-missing.png')

- It seems, the C0 values is not matching with the mean of the wind profile. 
- To examine where the problem is, we compared our script with python function from numpy.polynomial.chebyshev import chebfit

In [19]:
from numpy.polynomial.chebyshev import chebfit
CPtype = 1
ref_H = np.array([0, 10] + list(range(100, 501, 25)))   # Reference heights starts from 0 m.
Hn = normalize(Z, ref_H)
print('Reference height: ', ref_H)
print('Height: ', Z)
print('Normalized height: ', Hn)
for poly_order in range(1, 11):
    Coeff1 = Chebyshev_Coeff(Z, M.values,poly_order=poly_order, CPtype=CPtype, ref_H=ref_H)
    Coeff2 = chebfit(Hn, M, poly_order)    
    print(f'CPtype: {CPtype}, \t Poly order: {poly_order}, \t C0 from my script: {Coeff1[0]}, \t C0 from numpy function: {Coeff2[0]},\t mean wind: {M.mean().values}')

Reference height:  [  0  10 100 125 150 175 200 225 250 275 300 325 350 375 400 425 450 475
 500]
Height:  [ 10 100 125 150 175 200 225 250 275 300 325 350 375 400 425 450 475 500]
Normalized height:  [-0.96 -0.6  -0.5  -0.4  -0.3  -0.2  -0.1   0.    0.1   0.2   0.3   0.4
  0.5   0.6   0.7   0.8   0.9   1.  ]
CPtype: 1, 	 Poly order: 1, 	 C0 from my script: 5.00672907185047, 	 C0 from numpy function: 5.0067290718504704,	 mean wind: 5.134591579437256
CPtype: 1, 	 Poly order: 2, 	 C0 from my script: 4.722080496279246, 	 C0 from numpy function: 4.722080496279249,	 mean wind: 5.134591579437256
CPtype: 1, 	 Poly order: 3, 	 C0 from my script: 4.7030474418904875, 	 C0 from numpy function: 4.7030474418904875,	 mean wind: 5.134591579437256
CPtype: 1, 	 Poly order: 4, 	 C0 from my script: 4.699786384662939, 	 C0 from numpy function: 4.699786384662942,	 mean wind: 5.134591579437256
CPtype: 1, 	 Poly order: 5, 	 C0 from my script: 4.711025846050129, 	 C0 from numpy function: 4.711025846050127,	 m

In [64]:
import numpy as np
import xarray as xr

# Define the station ID and open the dataset
station_id = 'PROF_OWEG'
ds = xr.open_dataset(f'data/NYSM_standard_and_profiler_combined_wind_speed/{station_id}.nc')
data = ds.wind_speed.sel(range=slice(10, 500))

# Define the number of required non-missing points for intermediate range
intermediate_non_missing_points = 4

# Masks to ensure non-missing points in each segment
first_segment_mask = data.sel(range=slice(100, 200)).notnull().sum(dim='range') >= 2
last_segment_mask = data.sel(range=slice(400, 500)).notnull().sum(dim='range') >= 2
intermediate_segment_mask = data.sel(range=slice(225, 375)).notnull().sum(dim='range') >= intermediate_non_missing_points

# Combine masks to select data meeting all conditions
conditional_non_missing_mask = first_segment_mask & last_segment_mask & intermediate_segment_mask
filtered_data = data.where(conditional_non_missing_mask, drop=True)

# Within filtered data, identify all non-missing and some-missing instances
all_non_missing_mask = filtered_data.notnull().all(dim='range')
all_non_missing_data = filtered_data.where(all_non_missing_mask, drop=True)
some_missing_mask = filtered_data.isnull().any(dim='range')
some_missing_data = filtered_data.where(some_missing_mask, drop=True)

# Verifying if the data satisfies the conditions
print("Total instances after conditional filtering:", filtered_data.time.size)
print("Total non-missing instances after conditional filtering:", all_non_missing_data.time.size)
print("Total some missing instances after conditional filtering:", some_missing_data.time.size)

# randomly select one profile for visualization
np.random.seed(14000)
time_idx = np.random.choice(some_missing_data.time)
M = some_missing_data.sel(time=time_idx)
Z = M.range.values

Total instances after conditional filtering: 31605
Total non-missing instances after conditional filtering: 29581
Total some missing instances after conditional filtering: 2024


Bias: -0.0000
MAE: 0.1413
RMSE: 0.1791
Rmax: 0.3507


In [74]:
ref_H = np.array([0,10] + list(range(100, 501, 25)))
poly_order = 4
CPtype = 1
Coeff = Chebyshev_Coeff(Z, M.values,poly_order=poly_order, CPtype=CPtype, ref_H=ref_H)
Mp = WindProfile(Z,Coeff, poly_order=poly_order,CPtype=CPtype,ref_H=ref_H)
# Calculate statistics
Bias = np.mean(M - Mp)
MAE = np.mean(np.abs(M - Mp))
RMSE = np.sqrt(np.mean((M - Mp)**2))
Rmax = np.max(np.abs(M - Mp))

# Print the statistics
print(f'Bias: {Bias:.4f}')
print(f'MAE: {MAE:.4f}')
print(f'RMSE: {RMSE:.4f}')
print(f'Rmax: {Rmax:.4f}')

# Create Coeff_tot array
Coeff_tot = np.zeros(10)
Coeff_tot[:len(Coeff)] = Coeff

# Plot and save the figure
fig, ax = plt.subplots()
ax.plot(M, Z, 'or', markerfacecolor='r')
ax.plot(Mp, Z, '-k', linewidth=2)
ax.set(xlabel='Wind Speed (m/s)', ylabel='Height (m)')
ax.set_xlim([0, 30])
ax.set_ylim([0, 500])
ax.tick_params(labelsize=14)
ax.legend(['Measured', 'Modeled'])
plt.text(20, 400, f'C0 = {Coeff_tot[0]:.2f}')
plt.text(20, 350, f'C1 = {Coeff_tot[1]:.2f}')
plt.text(20, 300, f'C2 = {Coeff_tot[2]:.2f}')
plt.text(20, 250, f'C3 = {Coeff_tot[3]:.2f}')
plt.text(20, 200, f'C4 = {Coeff_tot[4]:.2f}')
plt.text(20, 150, fr'mean$_{{measured}}$ = {M.mean():.3f}')
plt.text(20, 100, fr'mean$_{{modelled}}$ = {Mp.mean():.3f}')
plt.savefig(f'data/Figures/chebyshev_fit_with_p{poly_order}_conditional-missing.png')

In [71]:
filtered_data

<xarray.DataArray 'wind_speed' (range: 18, time: 31605)> Size: 2MB
array([[ 3.5617554 ,  4.526838  ,  4.987547  , ...,  1.5772706 ,
         1.5513757 ,  0.77550125],
       [ 8.531982  ,  9.550075  , 11.010124  , ...,  2.1319165 ,
         1.4611508 ,  1.7538258 ],
       [ 9.215181  , 10.099037  , 11.522149  , ...,  1.3284127 ,
         0.8576386 ,  0.90296966],
       ...,
       [14.876497  , 14.761024  , 17.368385  , ...,  4.6811323 ,
         6.0455675 ,  5.7755823 ],
       [15.36315   , 14.560764  , 17.591682  , ...,  5.093437  ,
         6.3499765 ,  5.872446  ],
       [15.547368  , 14.981998  , 17.96914   , ...,  5.464223  ,
         6.607046  ,  5.9362464 ]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 253kB 2018-01-04T17:00:00 ... 2024-01-01
  * range    (range) int64 144B 10 100 125 150 175 200 ... 400 425 450 475 500
    station  <U9 36B 'PROF_OWEG'

In [84]:
ref_H = np.array([0,10] + list(range(100, 501, 25)))
poly_order = 4
CPtype = 1

# Number of observations, time steps, and height levels
n_height,n_time = filtered_data.shape

# Initialize Coeff array
Coeff = np.zeros((n_time, poly_order+1))
# Iterate over time steps
U = filtered_data.values
stime = time.time()
for t in range(n_time):
    Coeff[t, :] = Chebyshev_Coeff(Z, U[:,t],poly_order=poly_order, CPtype=CPtype, ref_H=ref_H)
etime = time.time()
print(f'Time elapsed: {etime-stime}s')
coeff_da = xr.DataArray(Coeff, dims=['time', 'coeff'], coords={'time': filtered_data.time, 'coeff': np.arange(poly_order+1)},name='Chebyshev_Coefficients')

# Save the coefficients to a netCDF file
output_dir = f'Data/Profiler_Chebyshev_Coefficients'
os.makedirs(output_dir, exist_ok=True)
coeff_da.to_netcdf(f'{output_dir}/{station_id}.nc')

Time elapsed: 3.97575306892395s
